In [39]:
import plotly.express as plt 
import pandas as pd
import subprocess
import os
import io
import numpy as np
import sys 
sys.path.append('./modules/')
from mva import *
import simulation

In [40]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))

In [41]:
mva = MVA(matrix,[5000,210,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)

def mvaToDataframe(matrix: np.ndarray)-> pd.DataFrame:
    vals = pd.DataFrame()
    vals["DELAY"] = matrix[0]
    vals["SWAP_IN"] = matrix[1]
    vals["CPU"] = matrix[2]
    vals["IO1"] = matrix[3]
    vals["IO2"] = matrix[4]
    vals["N"] = [i for i in range(0,len(matrix[0]))]
    return vals

mva()

In [42]:
#utilization = pd.read_csv("../../build/Scheduler/utilizations.csv",sep=";")
#throughputs = pd.read_csv("../../build/Scheduler/throughputs.csv", sep=";")
#meanWaits = pd.read_csv("../../build/Scheduler/meanWaits.csv",sep=";")
#meanClients = pd.read_csv("../../build/Scheduler/meanClients.csv", sep=";")

utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

visits = {}
visits["CPU"] = 250
visits["SWAP_IN"] = 2.5
visits["IO1"] = 16.25
visits["IO2"] = 6.25 

In [43]:
print(meanWaits)

     DELAY     SWAP_IN       CPU        IO1          IO2   N
0      0.0    0.000000  0.000000   0.000000     0.000000   0
1   5000.0  210.000000  2.700000  40.000000   180.000000   1
2   5000.0  223.824451  2.928527  43.260188   205.391850   2
3   5000.0  238.389124  3.177571  46.793396   235.823993   3
4   5000.0  253.533652  3.446068  50.579796   272.282610   4
5   5000.0  269.032970  3.731638  54.581095   315.853912   5
6   5000.0  284.600954  4.030402  58.738515   367.680155   6
7   5000.0  299.904587  4.336961  62.973254   428.890837   7
8   5000.0  314.589239  4.644616  67.190305   500.511697   8
9   5000.0  328.312511  4.945852  71.285815   583.361328   9
10  5000.0  340.780878  5.233042  75.157177   677.952217  10
11  5000.0  351.781406  5.499257  78.714047   784.416577  11
12  5000.0  361.201359  5.739004  81.887950   902.474954  12
13  5000.0  369.031668  5.948742  84.638366  1031.456793  13
14  5000.0  375.354754  6.127068  86.954192  1170.369616  14
15  5000.0  380.321190  

In [44]:


activeTimes = pd.DataFrame()
activeTimes["N"] = meanWaits["N"]
activeTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
activeTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
activeTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
activeTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]
print(activeTimes)


     N          CPU     SWAP_IN          IO1           IO2
0    0     0.000000    0.000000     0.000000      0.000000
1    1   675.000000  525.000000   650.000000   1125.000000
2    2   732.131662  559.561129   702.978057   1283.699060
3    3   794.392798  595.972811   760.392684   1473.899954
4    4   861.517004  633.834130   821.921680   1701.766314
5    5   932.909505  672.582426   886.942799   1974.086948
6    6  1007.600472  711.502385   954.500871   2298.000967
7    7  1084.240242  749.761468  1023.315381   2680.567734
8    8  1161.154028  786.473098  1091.842463   3128.198104
9    9  1236.462960  820.781277  1158.394499   3646.008298
10  10  1308.260497  851.952195  1221.304120   4237.201358
11  11  1374.814216  879.453516  1279.103264   4902.603606
12  12  1434.751032  903.003398  1330.679193   5640.468464
13  13  1487.185540  922.579169  1375.373453   6446.604955
14  14  1531.766887  938.386884  1413.005626   7314.810103
15  15  1568.642694  950.802975  1443.824881   8237.5124

In [45]:
def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    #data = fig.to_image("png")
    #f = open("./Images/{}.png".format(name),"+wb")
    #f.write(data)
    #f.close()
    fig.show()
pass

In [46]:
produce_graph_and_save(utilization,"Utilizations","B/T")

In [47]:
produce_graph_and_save(meanWaits,"meanWaits","ms",True)

In [48]:
produce_graph_and_save(meanClients,"meanClients","n")

In [49]:
produce_graph_and_save(throughputs,"throughputs","C/T")

In [50]:
produce_graph_and_save(activeTimes,"active_times","ms",True)

In [51]:
totalActiveTime = pd.DataFrame()
totalActiveTime["N"] = activeTimes["N"]
totalActiveTime["ActiveTime"] = activeTimes["CPU"] + activeTimes["IO1"] + activeTimes["IO2"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

In [52]:
totalActiveTime["ActiveTime"] = (meanClients["CPU"] + meanClients["IO1"] + meanClients["IO2"])/throughputs["SWAP_IN"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

# Markov chain model

In [53]:
mva = MVA(matrix,[5000,0,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)
mva()
utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

activeTimes = pd.DataFrame()
activeTimes["N"] = meanWaits["N"]
activeTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
activeTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
activeTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
activeTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]


In [54]:
produce_graph_and_save(utilization,"Utilizations_markov","B/T")

In [55]:
produce_graph_and_save(meanWaits,"meanWaits_markov","ms",True)

In [56]:
produce_graph_and_save(meanClients,"meanClients_markov","n")

In [57]:
produce_graph_and_save(throughputs,"throughputs_markov","C/T")

In [58]:
totalActiveTime = pd.DataFrame()
totalActiveTime["N"] = activeTimes["N"]
totalActiveTime["ActiveTime"] = activeTimes["CPU"] + activeTimes["IO1"] + activeTimes["IO2"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

In [59]:
totalActiveTime["ActiveTime"] = (meanClients["CPU"] + meanClients["IO1"] + meanClients["IO2"])/throughputs["SWAP_IN"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

In [60]:
scheduler_command = "../../build/Scheduler/scheduler"


In [61]:
def normalize_cpu_data(data: pd.DataFrame):
    copy = data
    view = copy[(copy["Station"] == "CPU") & (copy["Measure"] == "meanwaits")]
    view["R"] = view["R"]/10
    view["LB"] = view["LB"]/10
    view["HB"] = view["HB"]/10
    for index in view["R"].index:
        copy["R"][index] = view["R"][index]
        copy["LB"][index] = view["LB"][index]
        copy["HB"][index] = view["HB"][index]
        pass
    return copy
pass


In [62]:
simulator = simulation.SimulatorCommander("C:/Users/matteo.ielacqua/OneDrive - INPECO SPA/Desktop/Personal/NextEventSimulator/build/Scheduler/scheduler.exe")
#simulator = simulation.SimulatorCommander(scheduler_command)
def fetch_data(iterations: int, scenario: str):
    seed = 123456789
    simulator.scenario = scenario
    data = pd.DataFrame()
    for i in range(iterations):
        simdata = pd.DataFrame()
        simulator.seed = seed+i
        p=simulator.run()
        p.wait()
        out,err = p.communicate()
        simdata = pd.read_csv(io.BytesIO(b'Station;Measure;R;LB;HB;Samples;Precision;Expected\n' + out),encoding="utf8",sep=";")
        simdata["Simulation"] = [i+1]*len(simdata)
        simdata["Seed"] = [seed+i]*len(simdata)
        data = pd.concat([data,simdata],ignore_index=True)
        pass
    normalize_cpu_data(data)
    if not os.path.exists("./data/{}.csv".format(scenario)):
        data.to_csv("./data/{}.csv".format(scenario),sep=';')
        pass
    return data


Because cpu clients are considered to be leaving only when abandoning the CPU, the result is that the $\bar{\hat{w}} >> \bar{w}$ , since the latter one is calculated from MVA. A good method to "normalize" the data should be to consider $\bar{\hat{w}}(\frac{\bar{b}}{\delta})$ (biased_avg_wait,burst,time_slice)  so that the part in which the clients are waiting in the ready queue is removed. Since $\frac{\bar{b}}{\delta} = 10$ this is inserted as constant in the function

In [63]:
nompdscenario = pd.read_csv("./data/Default_NOMPD.csv",sep=";",index_col=0)
normalize_cpu_data(nompdscenario)
print(nompdscenario)

        Station      Measure            R           LB           HB  Samples  \
0           CPU  meanclients     1.460662     1.409468     1.511856       43   
1           CPU    meanwaits     6.527666     6.306251     6.749080       43   
2           IO1  meanclients     1.358137     1.330952     1.385321       43   
3           IO1    meanwaits    93.352304    92.074117    94.630491       43   
4           IO2  meanclients    11.835581    11.708681    11.962482       43   
..          ...          ...          ...          ...          ...      ...   
175         IO2  meanclients    12.020226    11.949170    12.091283       40   
176         IO2    meanwaits  2173.853337  2155.008956  2192.697719       40   
177     SWAP_IN  meanclients     0.846724     0.829862     0.863586       40   
178     SWAP_IN    meanwaits   385.659320   379.411253   391.907386       40   
179  ActiveTime    cycleTime  6894.395668  6622.931451  7165.859885       40   

     Precision     Expected  Simulation

/tmp/ipykernel_22520/227020074.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_22520/227020074.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_22520/227020074.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2252

In [64]:
copy = nompdscenario
hitted = ((copy["LB"] < copy["Expected"]) & (copy["Expected"] < copy["HB"]))
copy["Hitted"] = hitted

In [65]:
def hitpercentage(data:pd.DataFrame,station,measure):
    view = data[(data["Station"] == station)& (data["Measure"] == measure)]
    return (view["Hitted"].sum()/20)*100
    pass

for station in simulator.stationToRecord:
    print("Station{} Measure {} HitPercentage {}".format(station,"meanwaits",hitpercentage(copy,station,"meanwaits")))
    print("Station{} Measure {} HitPercentage {}".format(station,"meanclients",hitpercentage(copy,station,"meanclients")))
    pass

StationCPU Measure meanwaits HitPercentage 85.0
StationCPU Measure meanclients HitPercentage 80.0
StationIO1 Measure meanwaits HitPercentage 75.0
StationIO1 Measure meanclients HitPercentage 70.0
StationIO2 Measure meanwaits HitPercentage 70.0
StationIO2 Measure meanclients HitPercentage 70.0
StationSWAP_IN Measure meanwaits HitPercentage 90.0
StationSWAP_IN Measure meanclients HitPercentage 90.0


In [66]:
activetimes = copy[copy["Station"] == "ActiveTime"]
hits = activetimes["Hitted"].sum()
print("Activetimes hitted {}%".format(hits/20*100))

Activetimes hitted 90.0%


In [67]:
#Default = fetch_data(10,"Default")
#NegExp = fetch_data(10,"NegExpCpu")
#LTCpu = fetch_data(10,"LTCpu")
#NegExpLt = fetch_data(10,"NegExpLt")


Default = pd.read_csv("./data/Default.csv",sep=";",index_col=0)
NegExp = pd.read_csv("./data/NegExpCpu.csv",sep=";",index_col=0)
LTCpu = pd.read_csv("./data/LTCpu.csv",sep=";",index_col=0)
NegExpLt = pd.read_csv("./data/NegExpLt.csv",sep=";",index_col=0)




Default case

In [68]:
print(Default)

       Station      Measure            R           LB           HB  Samples  \
0          CPU  meanclients     1.350693     1.318493     1.382893       53   
1          CPU    meanwaits     6.132969     5.993446     6.272492       53   
2          IO1  meanclients     1.268768     1.245220     1.292316       53   
3          IO1    meanwaits    88.566038    87.442664    89.689412       53   
4          IO2  meanclients     6.523766     6.465629     6.581903       53   
..         ...          ...          ...          ...          ...      ...   
85         IO2  meanclients     6.532365     6.491643     6.573087       40   
86         IO2    meanwaits  1193.373144  1183.033604  1203.712684       40   
87     SWAP_IN  meanclients     0.829798     0.811632     0.847963       40   
88     SWAP_IN    meanwaits   378.493775   372.522734   384.464816       40   
89  ActiveTime    cycleTime  4246.976687  4105.746935  4388.206439       40   

    Precision     Expected  Simulation       Seed  

In [76]:
def print_active_time(data: pd.DataFrame, scenario):
    view = data[data["Station"] == "ActiveTime"]
    fig= plt.line(view,x="Simulation", y=["LB","HB","R","Expected"],title="{} Scenario: Mean Active Time".format(scenario))
    fig.update_yaxes(title_text="Mean Active Time(ms)")
    return fig
print_active_time(nompdscenario,"Default_NOMPD")

In [77]:
print_active_time(Default,"Default")

In [70]:
print_active_time(NegExp,"NegExp")

In [71]:
print_active_time(LTCpu,"LTCpu")

In [72]:
print_active_time(NegExpLt,"NegExpLt")

In [73]:
activeTimesR = pd.DataFrame()
def loadActiveTime(data:pd.DataFrame,scenario):
    activeTimesR[scenario] = data[data["Station"] == "ActiveTime"]["R"]
    pass

activeTimesR["Simulation"] = Default[Default["Station"] == "ActiveTime"]["Simulation"]
activeTimesR["Expected"] = Default[Default["Station"] == "ActiveTime"]["Expected"]

loadActiveTime(Default,"Default")
loadActiveTime(NegExp,"NegExp")
loadActiveTime(NegExpLt,"NegExpLt")
loadActiveTime(LTCpu,"LTCpu")
loadActiveTime(nompdscenario,"Default_NOMPD")
fig = plt.line(activeTimesR,x="Simulation",y=[x for x in activeTimesR.columns if x != "Simulation"],title="Mean active Time for scenario")
fig.update_yaxes(title_text="Mean active time(ms)")
fig.show()

Analysis of CPU

In [74]:
def print_cpu_measure(data: pd.DataFrame, scenario, measure):
    view = data[(data["Station"] == "CPU")& (data["Measure"] == measure)]
    fig = plt.line(view, x="Simulation" , y=["LB","HB","R","Expected"], title="{} CPU scenario {}".format(measure,scenario))
    return fig,view
    pass

fig,view = print_cpu_measure(nompdscenario,"Default_NOMPD","meanwaits")
fig.show()

In [78]:
print_cpu_measure(nompdscenario,"Default_NOMPD","meanclients")[0].show()

In [80]:
print_cpu_measure(Default,"Default","meanclients")[0]

In [81]:
print_cpu_measure(Default,"Default","meanwaits")[0]

In [83]:
print_cpu_measure(LTCpu,"LTCpu","meanwaits")[0]

In [84]:
print_cpu_measure(LTCpu,"LTCpu","meanclients")[0]


In [86]:
print_cpu_measure(NegExp,"NexExp","meanwaits")[0]

In [88]:
print_cpu_measure(NegExp,"NexExp","meanclients")[0]


In [90]:
print_cpu_measure(NegExpLt,"NegExpLt","meanwaits")[0]

In [91]:
print_cpu_measure(NegExpLt,"NegExpLt","meanclients")[0]

(Figure({
     'data': [{'hovertemplate': 'variable=LB<br>Simulation=%{x}<br>value=%{y}<extra></extra>',
               'legendgroup': 'LB',
               'line': {'color': '#636efa', 'dash': 'solid'},
               'marker': {'symbol': 'circle'},
               'mode': 'lines',
               'name': 'LB',
               'orientation': 'v',
               'showlegend': True,
               'type': 'scatter',
               'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
               'xaxis': 'x',
               'y': array([1.36203319, 1.33243143, 1.36202235, 1.34364405, 1.33696442, 1.36537804,
                           1.353924  , 1.3517582 , 1.35897287, 1.35420588]),
               'yaxis': 'y'},
              {'hovertemplate': 'variable=HB<br>Simulation=%{x}<br>value=%{y}<extra></extra>',
               'legendgroup': 'HB',
               'line': {'color': '#EF553B', 'dash': 'solid'},
               'marker': {'symbol': 'circle'},
               'mode': 'lines',
         